<a href="https://colab.research.google.com/github/Samruddhi-saoji/Travelling_salesman_problem/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt #to plot the graph
from random import random, shuffle, choices, choice
from numpy.random import randint
from copy import copy

### TSP class

In [2]:
#######################################################################
# cities = list of cities in the TSP
class TSP:
    #city = (x,y) point
    class City:
        def __init__(self, x, y, index, name) -> None:
            self.x = x
            self.y = y
            self.index = index #index of the city in the TSP city list
            self.name = name #name of the point

        #print(city) should print the name of the city
        def __repr__(self) -> str:
            return f"{self.name}"


    def __init__(self, n , city_list) -> None:
        self.n = n
        #city_list = [(x,y, name)]
        self.cities = [] #list of cities in the problem

        for c in city_list :
            x = c[0]
            y = c[1]
            index = c[2]
            name = c[3]
            self.cities.append(self.City(x,y,index, name))


    #return distance btw 2 cities
    def distance_btw(self, c1, c2):
        dist = np.sqrt((c1.x - c2.x)**2 + (c1.y - c2.y)**2)
        return dist


    # route = list of cities in order of visit
    def cost(self, route) :
        dist = 0
        n = self.n
        for i in range(0, n-1) :
            #add distance btw city i and city i+1 to the total distance
            city1 = route[i]
            city2 = route[i+1]
            dist = dist + self.distance_btw(city1, city2)

        #now only distance btw last city and first city is left
        dist = dist + self.distance_btw(route[n-1] , route[0])
        return dist


    #plot the graph
    def display(self, route) :
        x_coord = []
        y_coord = []
        names = []

        #add each city's coordinates to the list, in sequence
        for city in route:
            x_coord.append(city.x)
            y_coord.append(city.y)
            names.append(city.name)

        #add the first city again at the end
        x_coord.append(route[0].x)
        y_coord.append(route[0].y)
        names.append(route[0].name)

        #scatter plot
        plt.scatter(x_coord, y_coord)

        #label all points
        for i in range(len(x_coord)):
            plt.annotate(names[i], (x_coord[i], y_coord[i] + 0.2))

        #connect the points with lines
        plt.plot(x_coord, y_coord)
        plt.show()

In [16]:
####### creating a tsp ########
#randomly generate the cities
n = 15 #number of cities
cities = []
max_dist = 200
for i in range(n) :
    tup = (max_dist*random(), max_dist*random(), i, i+1) #(x, y, index, name)
    cities.append(tup)

#the TSP problem
tsp = TSP(n, cities)

## Different methods

### Simulated Annealing

### Genetic algorithm

#### Naive algo

In [ ]:
'''import numpy as np
from random import random, shuffle
from numpy.random import randint
from copy import copy'''


#######################################################################
class Chromosome:
    def __init__(self, n) -> None:
        self.n = n
        self.genes = [] #list of cities in order to be visited
        self.cost = None


    #print(chromosome) --> print the genes list
    def __repr__(self) -> str:
        return ''.join(str(gene) for gene in self.genes)




#####################################################################################

class GeneticAlgorithm1:
    def __init__(self, tsp, epochs, n,  pop_size, elitism_rate ,tournament_size, crossover_rate=0.8 ,mutation_rate=0.015) -> None:
        self.tsp = tsp
        self.epochs = epochs
        self.n = n

        self.pop_size = pop_size #same for all generations
        self.population = [] #list of individuals in the pop
        self.N = elitism_rate
        self.tournament_size = tournament_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate #prob of performing the crossover

        #randomly initialise the 0th generation
        #sample chromosome
        sample = tsp.cities

        #each individual of the pop will be shuffled version of sample
        for _ in range(0, pop_size) :
            individual = Chromosome(self.n)
            shuffle(sample)
            individual.genes = copy(sample)
            self.population.append(individual)
            individual.cost = tsp.cost(individual.genes)



    #the actual algorithm
    # return best oute, best distance
    def run(self):
        pop = self.population
        best = self.get_fittest(pop) #the best indi found yet
        max_fitness = self.fitness(best) #best fitness value found yet


        #while all possible states are not explored
        for _ in range(self.epochs):
            ##### selection and crossover #######
            next_gen = [] #list of individuals in next gen

            #elitism: N fittest individuals are directly passed down to the next gen
            elite = self.elitism(pop, self.N)
            next_gen.extend(elite)

            #select parents through tournament method and perform crossover
            tourn_size = self.tournament_size
            for i in range(0, self.pop_size - self.N):
                p1 = self.tournament(tourn_size, pop)
                child = self.crossover(p1)

                #mutate the child
                self.mutate(child) #cost is updated automatically if mutation occurrs

                #add child to next gen's population
                # child.cost = self.tsp.cost(child.genes)
                next_gen.append(child)


            #is the fittest of this new gen the best state yet?
            fittest = self.get_fittest(next_gen)
            if self.fitness(fittest) > max_fitness:
                best = fittest
                max_fitness = self.fitness(best)


            #next gen becomes the population
            pop = next_gen

        #training over
        # best individual = best
        best_dist = -1*max_fitness
        print("Total distance of this route is ", best_dist )
        return best.genes, best_dist
        #self.tsp.display(best)




    #fitness function
    def fitness(self, individual) :
        #high cost = low fitness
        return -1*individual.cost



    #return the fittest individual in the population
    def get_fittest(self, pop) :
        fittest = pop[0]
        max = self.fitness(fittest) #max fitness found yet

        for individual in pop :
            fitness = self.fitness(individual)

            if fitness > max:
                #this is the fittest chromosome yet
                fittest = individual
                max = fitness

        return fittest



    #return the top N fittest individuals in the pop
    def elitism(self,pop, N):
        population = copy(pop)
        #sort population in decreasing order of fitness
        population.sort(key = lambda individual : self.fitness(individual), reverse = True)
        #return first N elements
        return population[:N]



    #selection (tournament method)
        #tournament size = number of participants
    def tournament(self, tournament_size, pop):
        participants = []

        #select the particpants randomly from the population
        for _ in range(0, tournament_size):
            i = randint(self.pop_size)
            participants.append( pop[i] )

        #return the fittest participant
        return self.get_fittest(participants)




    #crossover  (1 parent 1 child approach)
    def crossover(self, p1):
        child = Chromosome(p1.n)
        pcost = p1.cost #parents cost
        prob = self.crossover_rate

        if random() < prob:
            #for two random indexes i1 and i2 (not inclusive)
            #reverse order of all elements btw i1 and i2
            i1 = randint(0,self.n)
            i2 = randint(0,self.n)

            if i1>i2:
                #exchange
                i1,i2 = i2, i1

            child.genes.extend(p1.genes[:i1+1])
            rem = copy(p1.genes[i1+1:i2])
            rem.reverse()
            child.genes.extend(rem)
            child.genes.extend(p1.genes[i2:])

            # shortcut: calculate child cost directly from parent cost
            if i1==i or i1==0 or i2==self.n-1:
                return p1 #child.cost = self.tsp.cost(child.genes)
            else:
                removed_paths = self.tsp.distance_btw(p1.genes[i1], p1.genes[i1+1]) + self.tsp.distance_btw(p1.genes[i2-1], p1.genes[i2])
                added_paths = self.tsp.distance_btw(child.genes[i1], child.genes[i1+1]) + self.tsp.distance_btw(child.genes[i2-1], child.genes[i2])
                ccost = pcost - removed_paths + added_paths
                child.cost = ccost
                #if ccost <= 0: print("parent cost: ", p1.cost, "\nDiscarded costs: ", removed_paths, "\nAdded paths: ", added_paths, "\nRoute: ", child.genes)

            return child

        return p1


    #mutation
    #swap the genes at 2 random indexes
    def mutate(self, chromosome):
        #change the gene mutation_rate% of times
        if random() < self.mutation_rate :
            i1 = randint(0,self.n)
            i2 = randint(0,self.n)
            chromosome.genes[i1] , chromosome.genes[i2] = chromosome.genes[i2] , chromosome.genes[i1]

            # update child cost
            chromodome.cost = self.tsp.cost(chromosome.genes)

#### Advanced version: evolved population

In [18]:
from random import sample

#######################################################################
class Chromosome:
    def __init__(self, n) -> None:
        self.n = n
        self.genes = [] #list of cities in order to be visited
        self.cost = None


    #print(chromosome) --> print the genes list
    '''def __repr__(self) -> str:
        return ''.join(str(gene) for gene in self.genes)'''




#####################################################################################

class GeneticAlgorithm2:
    def __init__(self, tsp, epochs, n,  pop_size, elitism_rate ,tournament_size, crossover_rate) -> None:
        self.tsp = tsp
        self.epochs = epochs
        self.n = n

        self.pop_size = pop_size #same for all generations
        self.population = [] #list of individuals in the pop
        self.N = elitism_rate
        self.tournament_size = tournament_size
        #self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate #prob of performing the crossover

        # what is the average distance for this tsp?
        # tot no of edges = n^2
        self.avg_dist = self.get_avg_cost(epochs=3*self.n)

        # get the advanced (evolved) gen0
        self.population = self.get_advanced_gen0()



    # get the cost of the average solution
    def get_avg_cost(self, epochs):
        total_dist = 0

        for e in range(epochs):
            # select two random cities
            c1, c2 = sample(self.tsp.cities, 2)
            dist = self.tsp.distance_btw(c1, c2)
            total_dist += dist

        avg_dist = total_dist / epochs #avg distance btw any 2 cities c1 and c2 in the tsp

        # TF: the cost for an avg route in the tsp
        # avg_cost = avg_dist * tot no of distances in a route
        # tot no of distances in a route = no of cities (as we are returning to the first city)
        avg_cost = avg_dist * self.n

        return avg_cost



    # evolve the gen 0 to get an advanced gen 0
    def get_advanced_gen0(self):
        advanced_pop = []

        def evolve_population():
            # step 1: get the population
            pop = []
            sample = tsp.cities # each indi will be a shuffled version of the sample
            for _ in range(0, self.pop_size) :
                individual = Chromosome(self.n)
                shuffle(sample)
                individual.genes = copy(sample)
                pop.append(individual)


            # step 2: add all above-avg individuals (less than avg cost) to advanced_pop
            for individual in pop:
                cost = self.tsp.cost(individual.genes)
                if cost <= self.avg_dist:
                    advanced_pop.append(individual)
                    individual.cost = cost

        # main code
        while len(advanced_pop) < self.pop_size:
            evolve_population()
        # now len(advanced_pop) > self.pop_size
        # return the (pop size) fittest individuals
        advanced_pop.sort(key = lambda individual : individual.cost, reverse = False)
        advanced_pop = advanced_pop[:self.pop_size]

        return advanced_pop




    #the actual algorithm
    # returns the best route, best dist
        # best route = [(x1, y1, z1) , (x2, y2, z2) ... ]
    def run(self):
        pop = self.population
        best = self.get_fittest(pop) #the best individual found yet
        max_fitness = self.fitness(best) #best fitness value found yet


        #while all possible states are not explored
        for _ in range(self.epochs):
            ##### selection and crossover #######
            next_gen = [] #list of individuals in next gen

            #elitism: N fittest individuals are directly passed down to the next gen
            elite = self.elitism(pop, self.N)
            next_gen.extend(elite)

            #select parents through tournament method and perform crossover
            tourn_size = self.tournament_size
            for i in range(0, self.pop_size - self.N):
                p1 = self.tournament(tourn_size, pop)
                child = self.crossover(p1)

                # mutate below average child
                if child.cost > self.avg_dist:
                    self.mutate(child)
                    child.cost = self.tsp.cost(child.genes)


                #add child to next gen's population
                next_gen.append(child)


            #is the fittest of this new gen the best state yet?
            fittest = self.get_fittest(next_gen)
            if self.fitness(fittest) > max_fitness:
                best = fittest
                max_fitness = self.fitness(best)

            #next gen becomes the population
            pop = next_gen

        #training over
        # best individual = best
        best_dist = -1*max_fitness
        print("Total distance of this route is ", best_dist)

        return best.genes, best_dist




    #fitness function
    def fitness(self, individual) :
        #cost = self.tsp.cost(genes)
        #high cost = low fitness
        return -1*individual.cost



    #return the fittest individual in the population
    def get_fittest(self, pop) :
        fittest = pop[0]
        max = self.fitness(fittest) #max fitness found yet

        for individual in pop :
            fitness = self.fitness(individual)

            if fitness > max:
                #this is the fittest chromosome yet
                fittest = individual
                max = fitness

        return fittest



    #return the top N fittest individuals in the pop
    def elitism(self,pop, N):
        population = copy(pop)
        #sort population in decreasing order of fitness
        population.sort(key = lambda individual : self.fitness(individual), reverse = True)
        #return first N elements
        return population[:N]



    #selection (tournament method)
        #tournament size = number of participants
    def tournament(self, tournament_size, pop):
        participants = []

        #select the particpants randomly from the population
        for _ in range(0, tournament_size):
            i = randint(self.pop_size)
            participants.append( pop[i] )

        #return the fittest participant
        return self.get_fittest(participants)



    #crossover  (1 parent 1 child approach)
    def crossover(self, p1):
        child = Chromosome(p1.n)
        pcost = p1.cost #parents cost
        prob = self.crossover_rate

        if random() < prob:
            #for two random indexes i1 and i2 (not inclusive)
            #reverse order of all elements btw i1 and i2
            i1 = randint(0,self.n)
            i2 = randint(0,self.n)

            if i1>i2:
                #exchange
                i1,i2 = i2, i1

            child.genes.extend(p1.genes[:i1+1])
            rem = copy(p1.genes[i1+1:i2])
            rem.reverse()
            child.genes.extend(rem)
            child.genes.extend(p1.genes[i2:])

            # calculate child cost
            if i1==i2 or i1==0 or i2==self.n-1:
                return p1 #child.cost = self.tsp.cost(child.genes)
            else:
                removed_paths = self.tsp.distance_btw(p1.genes[i1], p1.genes[i1+1]) + self.tsp.distance_btw(p1.genes[i2-1], p1.genes[i2])
                added_paths = self.tsp.distance_btw(child.genes[i1], child.genes[i1+1]) + self.tsp.distance_btw(child.genes[i2-1], child.genes[i2])
                ccost = pcost - removed_paths + added_paths
                child.cost = ccost
                #if ccost <= 0: print("parent cost: ", p1.cost, "\nDiscarded costs: ", removed_paths, "\nAdded paths: ", added_paths, "\nRoute: ", child.genes)

            return child

        return p1



    #mutation
    #swap the genes at 2 random indexes
    def mutate(self, chromosome):
        #change the gene mutation_rate% of times
        #if random() < self.mutation_rate :
        i1 = randint(0,self.n)
        i2 = randint(0,self.n)
        chromosome.genes[i1] , chromosome.genes[i2] = chromosome.genes[i2] , chromosome.genes[i1]

### A-star algorithm

#### Code

In [13]:
import heapq
import numpy as np

class Node:
    def __init__(self, state, n, parent, current):
        self.state = state #list of cities visited yet
        self.dist = 0 #distance coveres yet
        self.h = 0 # H(x) value
        self.n = n #no of cities visited yet
        self.parent = parent
        self.current = current #last visited city
        self.f = 0 # f(x) value

    def __lt__(self, other):
        return self.f < other.f


class A_star:
    def __init__(self, tsp):
        self.tsp = tsp


    #returns distance between 2 cities
    def distance_btw(self, c1, c2):
        return self.tsp.distance_btw(c1, c2)


    def calculate_f(self, node):
        node.f = node.dist - node.h


    # g(x) = distance covered
    def calculate_g(self, node):
        parent = node.parent
        last = parent.current #last visited city
        new = node.current #the newly visited city

        #New path added in route is the edge btw 'last' and 'new'
        node.dist = parent.dist + self.tsp.distance_btw(new, last)


    # H(x)
    def calculate_h(self, node):
        parent = node.parent
        new = node.current #the new city added to the route

        dist_eli = 0 #tot distance eliminated by adding 'new' city to the route

        for i in range(node.n - 1):
            city = node.state[i] #ith city in the route

            #add dist btw new city and ith city to eliminated distance
            dist_eli += self.tsp.distance_btw(city, new)
        node.h = parent.h + dist_eli


    ############################ solve TSP #######################
    #returns route, dist pair
    def run(self):
        solutions = [] #(route, dist) pairs
            #solution when starting from the ith city

        #e = 0 #iterations
        for i in range(self.tsp.n):
            #start from the city with the least no of neighbours
            start_city = self.tsp.cities[i]
            root = Node([start_city], 1, None, start_city)

            #priority queue (aka open list)
            queue = []
            heapq.heappush(queue, (root.f, root))

            # visit one city at atime
            while queue: #and e<epochs:
                #get the node with the lowest f value
                front = heapq.heappop(queue)[1]

                # if all cities have been visited, then tour is complete
                if front.n == self.tsp.n:
                    #revisit the starting city
                    front.state.append(start_city)
                    front.dist += self.distance_btw(front.current, start_city)

                    solutions.append((front.state, front.dist))
                    break

                # Which cities can be visited next?
                current_city = front.current #last visited city in the current state
                neighbours = [c for c in self.tsp.cities if c not in front.state]
                m = len(neighbours)

                #if the current city has no neighbours, then this node is a deadend
                if m == 0:
                    continue

                #create 'm' child nodes, each visiting one neighbour
                for i in range(m):
                    next_city = neighbours[i]

                    #generate child node
                    temp = Node(front.state + [next_city], front.n + 1, front, next_city)

                    #calculate f, g, h for child node
                    self.calculate_g(temp)
                    self.calculate_h(temp)
                    self.calculate_f(temp)

                    #push child node to queue
                    heapq.heappush(queue, (temp.f, temp))

                    #increment iteration number
                    #e = e + 1

        # check the best route found
        print("No of full routes complted = ", len(solutions))

        min_dist = solutions[0][1]
        min_route = solutions[0][0]

        for sol in solutions: #sol = (state, dist)
            if sol[1] < min_dist:
                #update
                min_dist = sol[1]
                min_route = sol[0]

        return min_route, min_dist

#### Testing

In [14]:
a_star = A_star(tsp)
route, dist = a_star.run()
print(route, dist)

No of full routes complted =  100
[15, 56, 89, 94, 19, 68, 65, 12, 7, 27, 39, 74, 17, 25, 70, 43, 95, 86, 60, 54, 66, 8, 44, 81, 96, 59, 69, 45, 6, 38, 33, 21, 90, 4, 22, 34, 67, 46, 13, 50, 48, 53, 92, 55, 83, 29, 26, 32, 58, 47, 1, 62, 80, 91, 72, 52, 76, 57, 42, 14, 2, 64, 16, 84, 63, 11, 9, 23, 77, 85, 35, 93, 75, 49, 18, 61, 82, 30, 73, 98, 24, 78, 99, 71, 88, 51, 41, 40, 10, 20, 37, 31, 36, 28, 100, 3, 97, 79, 87, 5, 15] 793802.1968369907


### Ant colony optimisation

#### Code

In [7]:
''' Note
- route = [C1, C2, C3, C4]; path = (C2, C3)
- thus, route = group/sequence of paths
'''

class ACO:
    def __init__(self, tsp, epochs, ants=10, alpha=1, beta=5, rho=0.5):
        # n = no of cities
        self.tsp = tsp
        self.ants = ants    #number of ants
        self.epochs = epochs
        self.alpha = alpha  # Pheromone weight
        self.beta = beta    # Heuristic weight
        self.rho = rho      #rate of evaporation
        self.pheromone = np.ones((tsp.n, tsp.n)) #2d matrix of shape (n,n)
            # pheromone[i][j] = amt of pheromone on the path btw city 'i' and city 'j'
            # i, j --> indexes of the cities in TSP cities list


    ######################### solve TSP #########################
    def run(self):
        #best solution found yet
        best_route = None
        best_distance = float('inf')

        #run the algorithm
        for _ in range(self.epochs):
            # for each ant
            for ant in range(self.ants):
                route = self.get_route() #route taken by this ant
                distance = self.tsp.cost(route) #dist btw last and 1st city is counted in tsp.cost() function

                #if this is the best path yet
                if distance < best_distance:
                    best_distance = distance
                    best_route = route

        return best_route, best_distance


    ########################## get the route #########################
    def get_route(self):
        ########### Step 1: start from random city #######
        current_city = choices(self.tsp.cities)[0]
        route = [current_city] #list of visited cities

        #list of unvisited cities
        remaining = set(self.tsp.cities)
        remaining.remove(current_city)

        ########### Step 2: visit one city at a time #######
        while remaining:
            #select the next city to visit based on f value
            next_city = self.select_next_city(current_city, remaining)

            # add it to the route, and remove from list of unvisited cities
            route.append(next_city)
            remaining.remove(next_city)

            #update
            current_city = next_city
        #all cities have been visited exactly once



        ########### Step 3: update pheromone level for this route #######
        # one route contains n-1 paths
        self.update_pheromone(route) #update for all paths

        return route


    ######################### select the next city/path #########################
    '''
        - ant is currently at city "current_city". it has to go to one of its neighbours.
        - neighbours = list of unvisited neighbour
        - the probability of selecting a neighbour as the next city to visit depends on its f value
        '''

    def select_next_city(self, current_city, neighbours):
        m = len(neighbours) #tot no. of unvisited neighbours

        ##### Step 1: calculate f values for each neighbour #####
        f_values = [] #same order as neighbours list

        for nbr in neighbours:
            # path = edge btw current_city and nbr
            h = self.pheromone[current_city.index][nbr.index] #pheromone
            d = self.tsp.distance_btw(current_city, nbr) #path length
            g = 1/d
            f = (h ** self.alpha)*(g ** self.beta)

            f_values.append(f)

        ##### Step 2: calculate probabilities for all neighbours #####
            # prob of nbr1 = f1/(f1 + f2 + .. fm)
            # where f1 = f val of nbr1
        f_values = np.array(f_values) #convert to array
        probabilities = f_values/f_values.sum()

        ##### Step 3: select next city based on f values #####
        next_city = np.random.choice(list(neighbours), p=probabilities)
        return next_city
        #index = np.random.choice(m, p=probabilities)
        #return list(neighbours)[index]


    ######################### update pheromone on path #########################
    def update_pheromone(self, route):
        dist = self.tsp.cost(route) #tot distance of the route

        #for each path (i,j) in the route
        for c in range(self.tsp.n - 1):
            #cities c1 and c2
            c1, c2 = route[c], route[c+1]
            i, j = c1.index, c2.index #indices a/c to TSP

            #calculate h (pheromone amt)
            h_old = self.pheromone[i][j]
            h_new = h_old*(1-self.rho) + (self.rho/dist)
                # rho/dist = amt of pheromone evaporated per unit time per unit length (dist)

            #update pheromone amt
            self.pheromone[i][j] = h_new
            self.pheromone[j][i] = h_new

#### Testing

In [8]:
#Hyper parameters
epochs = 50
ants = 10 #no of ants
alpha = 1
beta = 5
rho = 0.5

aco = ACO(tsp, epochs=epochs, ants=ants, alpha=alpha, beta=beta, rho=rho)
best_route, best_distance = aco.run()
print(f'Best route: {best_route}')
print(f'Best distance: {best_distance}')

KeyboardInterrupt: 

## Comparison

In [19]:
''' Genetic Algo'''
print("Genetic Algorithm")
#hyper parameters
epochs = 50
pop_size = 10
N = 2 #elitism rate
tournament_size = 3
crossover_rate = 0.8
mutation_rate = 0.015

algo = GeneticAlgorithm2(tsp, epochs, n, pop_size, N , tournament_size, crossover_rate )
algo.run()


''' A star Algo'''
print("\nA star algorithm")
epochs = 500
a_star = A_star(tsp)
route, dist = a_star.run()
print(dist)

''' Ant Colony Optimisation '''
print("\nAnt colony optimisation")
#Hyper parameters
epochs = 50
ants = 10 #no of ants
alpha = 1
beta = 5
rho = 0.5

aco = ACO(tsp, epochs=epochs, ants=ants, alpha=alpha, beta=beta, rho=rho)
best_route, best_distance = aco.run()
print(f'Best route: {best_route}')
print(f'Best distance: {best_distance}')

Genetic Algorithm
Total distance of this route is  977.3829142653301

A star algorithm
No of full routes complted =  15
1204.5188102899676

Ant colony optimisation
Best route: [6, 2, 14, 11, 10, 12, 15, 4, 7, 3, 9, 1, 13, 8, 5]
Best distance: 643.4247201434183
